#  Домашнее задание по NLP # 3 [100 баллов] 
## Классификация и генерация фамилий 

В этом домашнем задании вам предстоит классифицировать и генерировать фамилии на 19 разных языках. Обучающие данные хранятся в папке data и разбиты по языкам: один язык – одна подпапка. Ниже представлен код для считывания данных в словарь вида: 
```d{язык} : [список имен]```.

In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Activation, Embedding, Flatten, Dropout
from sklearn.utils import shuffle

C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM

In [7]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import numpy as np
import pandas as pd

In [8]:
from sklearn.model_selection import train_test_split
import nltk

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import *
from sklearn.feature_extraction.text import *
from sklearn.metrics import *
from sklearn.naive_bayes import MultinomialNB

In [10]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob

def findFiles(path): return glob.glob(path)

print(findFiles('../../../Data/NLP-HW3/names/*.txt'))


def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return lines

all_categories = []
category_lines = {}
for filename in findFiles('../../../Data/NLP-HW3/names/*.txt'):
    category = filename.split('/')[-1].split('.')[0][6:]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

print(category_lines['Russian'][100:120])

['../../../Data/NLP-HW3/names\\Arabic.txt', '../../../Data/NLP-HW3/names\\Chinese.txt', '../../../Data/NLP-HW3/names\\Czech.txt', '../../../Data/NLP-HW3/names\\Dutch.txt', '../../../Data/NLP-HW3/names\\English.txt', '../../../Data/NLP-HW3/names\\French.txt', '../../../Data/NLP-HW3/names\\German.txt', '../../../Data/NLP-HW3/names\\Greek.txt', '../../../Data/NLP-HW3/names\\Irish.txt', '../../../Data/NLP-HW3/names\\Italian.txt', '../../../Data/NLP-HW3/names\\Japanese.txt', '../../../Data/NLP-HW3/names\\Korean.txt', '../../../Data/NLP-HW3/names\\Polish.txt', '../../../Data/NLP-HW3/names\\Portuguese.txt', '../../../Data/NLP-HW3/names\\Russian.txt', '../../../Data/NLP-HW3/names\\Scottish.txt', '../../../Data/NLP-HW3/names\\Spanish.txt', '../../../Data/NLP-HW3/names\\Vietnamese.txt']
['Abels', 'Abelsky', 'Abeltsev', 'Abelyan', 'Aberson', 'Abertasov', 'Abesadze', 'Abezgauz', 'Abgaryan', 'Abibulaev', 'Abidoff', 'Abidov', 'Abih', 'Abikh', 'Abisaloff', 'Abisalov', 'Abitoff', 'Abitov', 'Abjaliloff

In [11]:

df = pd.read_csv('names.csv')
df.drop('Unnamed: 0', axis = 1, inplace=True)

## Часть 1. Предварительная обработка данных [10 баллов]

1. Удалите неоднозначные фамилии (т.е. одинаковые фамилии на разных языка), если такие есть;
2. Оцените 
* среднюю длину фамилии по всей коллекции
* по каждому языку
3. Для последующей классификации (части 2 и 3) оздайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы: то есть, в обучающее множество должно входить примерно одинаковое количество фамилий на разных ящыка. 

### Удалите неоднозначные фамилии (т.е. одинаковые фамилии на разных языка), если такие есть

In [12]:
df.shape

(20074, 2)

In [13]:
df = df.drop_duplicates('Name')

In [14]:
df.shape

(17458, 2)

### Оцените среднюю длину фамилии по всей коллекции

In [15]:
df['Len'] = df.Name.apply(len)

In [16]:
df.Len.mean()

7.366422270592278

### По каждому языку

In [17]:
df.groupby('Lang').mean()

,Len
Lang,
Arabic,5.592593
Chinese,3.674797
Czech,6.582505
Dutch,6.939929
English,6.413517
French,7.068182
German,6.810680
Greek,8.843750
Irish,7.341463


In [18]:
df = df[df['Lang'] != 'Scottish']

### Стратифицированное разбиение выборки

In [19]:
Y = df.Lang
X = df.Name

In [20]:
def lower(s):
    return(s.lower())

In [21]:
X = X.apply(lower)

In [22]:
from sklearn.model_selection import StratifiedShuffleSplit
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=777)

for train_index, test_index in splitter.split(X, Y):
    X_train = X.iloc[train_index]
    X_test = X.iloc[test_index]
    
    Y_train = Y.iloc[train_index]
    Y_test = Y.iloc[test_index]

In [23]:
len(X_train), len(Y_train), len(X_test), len(Y_test)

(13965, 13965, 3492, 3492)

##  Часть 2. Базовый метод классификации [20 баллов]



Используйте метод наивного Байеса, логистическую регрессию или любой другой метод для классификации фамилий: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:

In [20]:
from nltk.util import ngrams

In [ ]:
def get_ngrams(s):
    l = list(ngrams(s, 3))
    ngr_str = ''
    for i in range(len(l)):        
        ngr = ''
        for j in l[i]:
            ngr += j
        ngr_str += ngr
        ngr_str += ' '
    return(ngr_str)

In [ ]:
X_train = X_train.apply(get_ngrams)
X_test = X_test.apply(get_ngrams)

In [ ]:
clf = Pipeline([ 
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()), 
    ('clf', MultinomialNB())
])

In [ ]:
clf.fit(X_train, Y_train)

In [ ]:
Y_pred = clf.predict(X_test)

In [ ]:
true = Y_test
predictions = Y_pred
print("Precision: {0:6.2f}".format(precision_score(true, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(true, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(true, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(true, predictions)))
print(classification_report(true, predictions))

##  Часть 3. Нейронная сеть [35 баллов]


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс фамилии. 

Представление имени для классификации в этом случае: 
1 вариант: бинарная матрица размера (количество букв в корпусе $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$. То есть, используем one hot encoding для векторизации букв.  
2 вариант: Embedding'и символов. 

Выберите тот вариант, который вам проще или интереснее реализовать :) 

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами по accuracy, micro- и macro- F-measure, precision, recall. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

Если совсем не получается запрограммировать нейронную сеть самостоятельно, обратитесь к туториалу тут: https://github.com/divamgupta/lstm-gender-predictor

In [24]:
df.head()

,Lang,Name,Len
0,Arabic,Khoury,6
1,Arabic,Nahas,5
2,Arabic,Daher,5
3,Arabic,Gerges,6
4,Arabic,Nazari,6


In [25]:
max_len = df.Len.max()
max_len

20

In [26]:
X.shape

(17457,)

In [27]:
chars = set(  "".join(X))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [28]:
X = X.reset_index(drop=True)
Y = Y.reset_index(drop=True)

In [29]:
out_len = Y.nunique()

In [27]:
x = np.zeros((len(X) , max_len, len(chars) ), dtype=np.bool)
y = np.zeros((len(X) , Y.nunique() ), dtype=np.bool)


for i, name in enumerate(X):
    for t, char in enumerate(name):
        x[i, t, char_indices[char]] = 1
    y[i][Y.unique().searchsorted(Y[i])] = 1


In [30]:
print('Build model...')
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(max_len, len(chars))))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(out_len))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy', optimizer='Adam')

Build model...


In [ ]:
model.save_weights

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 20, 128)           95744     
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 17)                2193      
_________________________________________________________________
activation_1 (Activation)    (None, 17)                0         
Total params: 229,521
Trainable params: 229,521
Non-trainable params: 0
_________________________________________________________________


In [30]:
x_train, x_valid, y_train, y_valid = train_test_split(
                                                        x,
                                                        y,
                                                        shuffle=True,
                                                        train_size=0.8,
                                                        random_state=42
                                                        )

C:\Anaconda\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [31]:
model.fit(x_train, y_train, batch_size=16, epochs=4)

Epoch 1/4
13965/13965 [==============================] - 74s 5ms/step - loss: 0.1105
Epoch 2/4
13965/13965 [==============================] - 72s 5ms/step - loss: 0.0943
Epoch 3/4
13965/13965 [==============================] - 73s 5ms/step - loss: 0.0864
Epoch 4/4
13965/13965 [==============================] - 79s 6ms/step - loss: 0.0809


In [32]:
y_preds = model.predict(x_valid)

In [33]:
for i in range(len(y_preds)):
    max_ = 0
    for j in range(len(y_preds[i])):
        if y_preds[i][j] > max_:
            max_ = y_preds[i][j]
    for j in range(len(y_preds[i])):
        if y_preds[i][j] == max_:
            y_preds[i][j] = True
        else:
            y_preds[i][j] = False

In [34]:
true = y_valid
predictions = y_preds
print("Precision: {0:6.2f}".format(precision_score(true, predictions, average='macro')))
print("Recall: {0:6.2f}".format(recall_score(true, predictions, average='macro')))
print("F1-measure: {0:6.2f}".format(f1_score(true, predictions, average='macro')))
print("Accuracy: {0:6.2f}".format(accuracy_score(true, predictions)))

Precision:   0.25
Recall:   0.25
F1-measure:   0.23
Accuracy:   0.74


C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [35]:
model.save_weights('weights')

In [ ]:
!nvidia-smi

##  Часть 4. Генерация фамилии [35 баллов]

Используйте архитектуру нейронной сети из Части 3 для генерации имени. В этот момент можно забыть про разбиение коллекции на обучающее и тестовое множество. 

Рассмотрите два сценария генерация имени:
* обучение нейронной сети по всей коллекции
* обучение нейронной сети с обуславливанием на язык

Для обуславливания на язык нужно изменить векторное представление фамилии. До этого момента мы разбивали фамилию на отдельные символы и находили векторное представление каждого символа. Теперь добавим в начало фамилии метку языка и будем ее считать первым символов фамилии. Так нейронная сеть научиться понимать, на каком языке написана фамилия. 

Пример: ```[rus bos i v a n o v eos pad pad pad]```

Когда будем генерировать новую фамилию, будем начинать процесс генерации не с символа начала последовательности ```bos```, а с символа языка и символа начала последовательности ```rus bos```. 

Привидите несколько примеров удачно сгенерированных фамилий. 

In [ ]:
from keras.utils import to_categorical
import numpy as np

X_names = ['bos ' + ' '.join(name) for name in X]
Y_names = [' '.join(name) + ' eos' for name in X]
maxlen = max([len(name) for name in X])+1

In [ ]:
tokenizer = Tokenizer(num_words=len(alphabet)+2)
tokenizer.fit_on_texts(X_names+Y_names)

sequences = tokenizer.texts_to_sequences(X_names)
X_train = pad_sequences(sequences, maxlen=maxlen, padding='post')


sequences = tokenizer.texts_to_sequences(Y_names)
Y_train = pad_sequences(sequences, maxlen=maxlen,padding='post')
temp = Y_train.copy()

Y_train_cat  = [to_categorical(sent, num_classes=len(alphabet)+2) for sent in Y_train]
Y_train =  np.asarray(Y_train_cat)

In [ ]:
char_index = tokenizer.word_index
index_char = {i: c for c, i in char_index.items()}

In [ ]:
model = Sequential()

model.add(Embedding(len(alphabet)+2, 30, input_length=maxlen))
model.add(LSTM(128, return_sequences = True))

model.add(Dense(len(alphabet)+2, activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
for iteration in range(1, 20):
    X_train_shuffled, y_train_shuffled = shuffle(X_train, Y_train)
    model.fit(X_train_shuffled, y_train_shuffled, batch_size=len(X_train), epochs=1, verbose = 1)

In [ ]:
def sample(preds):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) #/ temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.choice(range(len(alphabet)+2), p = preds)
    return probas

In [ ]:
generated = ''
seed = 'bos'
generated += seed + ' '
print('----- Generating with seed: "' + seed + '"')
print(generated)


for i in range(7): 
    sequences = tokenizer.texts_to_sequences([seed])
    X_pred = pad_sequences(sequences, maxlen=maxlen, padding = 'post')

    preds = model.predict(X_pred, verbose=0)[0]
    samples = [sample(p) for p in preds]
    next_index = samples[i]
    while next_index == 0 or next_index == 10:
        samples = [sample(p) for p in preds]
        next_index = samples[i]
    if next_index > 55:
        next_index = 55
    next_char = index_char[next_index]
    generated += next_char + ' '
    print(generated)
    seed += next_char
    if next_char == 'eos':
        break
    